In [1]:
import sys
import os
import pandas as pd
from src.components.ETL_manager import DataIngestor
from src.components.ETL_manager import DataProcessor

input_file = r'/raw_file/915010018577756.csv'
input_file = os.getcwd()+input_file

# Example usage
try:
    df = DataIngestor.read_csv_files(
        file_path=input_file, 
        header_identifier='Tran Date', 
        footer_identifier='Charge breakup'
        )
except TypeError as e:
    raise(f'error occured: {e}')

df = DataProcessor.remap_transactions(data=df, transaction_col_name="PARTICULARS")

In [3]:
df.dtypes

Tran Date      object
CHQNO          object
PARTICULARS    object
DR             object
CR             object
BAL            object
SOL            object
dtype: object